In [2]:
import pandas as pd
import re
from imdb import IMDb
from tqdm import tqdm
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import requests
pd.options.display.max_columns = 49

In [3]:
title = pd.read_csv('data.tsv', delimiter='\t', encoding='utf-8')

C:\Users\pbren\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
a = list(set(title[title.types == 'original'].titleId.apply(lambda t: re.sub('tt', '', t))))

In [5]:
a[0]

'5619444'

In [65]:
# Define a function to perform both stemming and tokenization
def tokenize_and_lem(text):
    
    # Tokenize by sentence, then by word
    tokens = [word for word in nltk.word_tokenize(text)]
    
    # Filter out raw tokens to remove noise
    filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    
    # Stem the filtered_tokens
    lems = [lemmer.lemmatize(token) for token in filtered_tokens]
    
    return lems

In [66]:
# Create an English language SnowballStemmer object
lemmer = WordNetLemmatizer()

# Instantiate TfidfVectorizer object with stopwords and tokenizer
# parameters for efficient processing of text
tfidf_vectorizer = TfidfVectorizer()

In [69]:
movies = pd.DataFrame({'title':list(title[title.types == 'original'].title)[-5000:], 'titleID':a[-5000:], 'plot':movies_plot})
movies = movies[movies['plot'] != 'NaN']
movies['plot'] = movies['plot'].apply(lambda t: t[0])
movies.reset_index(inplace=True)

In [70]:
# Fit and transform the tfidf_vectorizer with the "plot" of each movie
# to create a vector representation of the plot summaries
tfidf_matrix = tfidf_vectorizer.fit_transform([x for x in movies['plot']])

print(tfidf_matrix.shape)

(3066, 19064)


In [71]:
# Import k-means to perform clusters
from sklearn.cluster import KMeans

# Create a KMeans object with 5 clusters and save as km
km = KMeans(n_clusters=5)

# Fit the k-means object with tfidf_matrix
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

# Create a column cluster to denote the generated cluster for each movie
movies['cluster'] = clusters

In [72]:
# Import cosine_similarity to calculate similarity of movie plots
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the similarity distance
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

sim = pd.DataFrame(similarity_distance)

res = list(sim.loc[181].sort_values(ascending=True).index[1:5])

for r in res:
    print(movies.title[r], movies.cluster[r])

In [80]:
movies.to_csv('movies_5000.csv')

In [ ]:
def find_movies(title, number):
    ind = movies[movies['title'] == title].index[0]

In [1]:
movies.head()

NameError: name 'movies' is not defined

## CLEAN PROJECT START

In [43]:
import urllib.request as request
import csv
url = 'https://raw.githubusercontent.com/zacharyang/movies-project/master/data/clean/OMdb_mojo_clean.csv'
#r = request.urlopen(url).read().decode('utf8').split("\n")
#reader = csv.reader(r)
#data = pd.read_csv(url, error_bad_lines=False)
data = pd.read_csv('data.csv', error_bad_lines=False, index_col=0)
data.drop_duplicates(subset='Plot', inplace=True)

### Creating clean df

In [44]:
data_processed = data.drop(['IMdb_ID','Plot','TMdb_id','Actors', 'DVD'], axis=1)

In [45]:
data_processed.head(2)

,Awards,BoxOffice,Country,Director,Genre,IMdb_score,Language,Metascore,Production,RT_score,Rated,Released,Runtime,Title,Type,Website,Year,actor_1,actor_2,awards,bo_year,bo_year_rank,director_1,director_2,domestic-gross,domestic-pct,imdbVotes,itunes_id,logBoxOffice,mojo_title,nominations,num_actor,num_director,num_genre,num_lang,oscar_noms,oscar_wins,overseas-gross,overseas-pct,search_strs,studio,titles,worldwide-gross,years
0,1 win & 1 nomination.,NaN,France,Georges Lautner,"Action, Crime, Drama, Thriller",76.0,French,78.117,NaN,78.117,R,21 Oct 1981,108.0,The Professional,movie,NaN,1981,Jean-Paul Belmondo,Jean Desailly,1.0,2012.0,75.0,Georges Lautner,NaN,49100000.0,57.5,11220.0,270815716,NaN,The Possession,1.0,4,1,4,1.0,0.0,0.0,36300000.0,42.5,The+Possession,LGF,The Professional,85400000.0,1998
1,Won 1 Oscar. Another 31 wins & 54 nominations.,363995937.0,"UK, USA",Jon Favreau,"Adventure, Drama, Family, Fantasy",74.0,English,77.000,Walt Disney Pictures,95.000,PG,15 Apr 2016,106.0,The Jungle Book,movie,http://movies.disney.com/the-jungle-book-2016,2016,Neel Sethi,Bill Murray,31.0,2016.0,5.0,Jon Favreau,NaN,364000000.0,37.7,231929.0,1098066757,8.561097,The Jungle Book 2016,54.0,4,1,4,1.0,0.0,1.0,602500000.0,62.3,The+Jungle+Book+2016,BV,The Jungle Book (2016),966600000.0,2016


### Processing awards

In [46]:
def extract_nom(sent, list_oscar, list_win, list_nom):
    """
    Extract oscar, win and nominations number
    """
    if type(sent) == float:
        list_oscar.append(0)
        list_win.append(0)
        list_nom.append(0)
    else:
        result_oscar = re.findall('[0-9]+ Oscar', sent)
        if len(result_oscar)>0:
            list_oscar.append(re.findall('[0-9]+', result_oscar[0])[0])
        else:
            list_oscar.append(0)
        result_win = re.findall('[0-9]+ win', sent)
        if len(result_win)>0:
            list_win.append(re.findall('[0-9]+', result_win[0])[0])
        else:
            list_win.append(0)
        result_nom = re.findall('[0-9]+ nom', sent)
        if len(result_nom)>0:
            list_nom.append(re.findall('[0-9]+', result_nom[0])[0])
        else:
            list_nom.append(0)
    return list_oscar, list_win, list_nom

In [47]:
oscar = []
win = []
nom = []

for t in data_processed.Awards:
    extract_nom(t, oscar, win, nom)

data_processed.drop(['Awards'], axis=1, inplace=True)
data_processed['Oscar'] = oscar
data_processed['Win'] = win
data_processed['Nom'] = nom

### Processing Genre

In [48]:
genres = [re.findall('[a-zA-Z-a-zA-Z]+',t) for t in data_processed.Genre]

In [49]:
genre = []
for t in genres:
    for z in t:
        genre.append(z)
genre = list(set(genre))

In [50]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

s = genres
test = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=data_processed.index)

In [51]:
data_processed = data_processed.join(test)

### Processing Released

In [52]:
date = []
for t in data_processed['Released']:
    try:
        date.append(int(re.findall('[0-9]{4}', t)[0]))
    except:
        date.append(t)

In [53]:
data_processed['Released'] = date

### Processing non numerical columns

In [54]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

In [55]:
data_processed.columns

Index(['BoxOffice', 'Country', 'Director', 'Genre', 'IMdb_score', 'Language',
       'Metascore', 'Production', 'RT_score', 'Rated', 'Released', 'Runtime',
       'Title', 'Type', 'Website', 'Year', 'actor_1', 'actor_2', 'awards',
       'bo_year', 'bo_year_rank', 'director_1', 'director_2', 'domestic-gross',
       'domestic-pct', 'imdbVotes', 'itunes_id', 'logBoxOffice', 'mojo_title',
       'nominations', 'num_actor', 'num_director', 'num_genre', 'num_lang',
       'oscar_noms', 'oscar_wins', 'overseas-gross', 'overseas-pct',
       'search_strs', 'studio', 'titles', 'worldwide-gross', 'years', 'Oscar',
       'Win', 'Nom', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short',
       'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')

In [56]:
data_processed[['Oscar', 'Win', 'Nom']] = data_processed[['Oscar', 'Win', 'Nom']].astype(int)

In [57]:
data_processed = pd.get_dummies(data_processed)

In [58]:
#data_processed.drop(['Title', 'Genre'], axis=1, inplace=True)
#data_processed.drop(['Genre'], axis=1, inplace=True)
data_processed.fillna(0, inplace=True)

In [59]:
non_num = list(data_processed.select_dtypes(exclude=[float, 'int64', 'int32']).dtypes.index)
data_le = data_processed[non_num]
le = OneHotEncoder(categories='auto')
#le.fit_transform(data_le)

In [60]:
le.fit(data_processed)

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [61]:
km = KMeans(n_clusters=7)
km.fit(data_processed)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [62]:
data_processed['km_result'] = list(km.labels_)

In [63]:
data['results'] = list(km.labels_)

In [64]:
tfidf_vectorizer = TfidfVectorizer()

In [65]:
tfidf_matrix = tfidf_vectorizer.fit_transform([x for x in collec['']])

In [66]:
# Import k-means to perform clusters
from sklearn.cluster import KMeans

# Create a KMeans object with 5 clusters and save as km
km = KMeans(n_clusters=5)

# Fit the k-means object with tfidf_matrix
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

# Create a column cluster to denote the generated cluster for each movie
data['cluster'] = clusters

In [72]:
data.shape

(477, 51)

In [70]:
# Import cosine_similarity to calculate similarity of movie plots
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the similarity distance
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

sim = pd.DataFrame(similarity_distance)

res = list(sim.loc[186].sort_values(ascending=True).index[1:10])

for r in res:
    print(data.titles[r], data.cluster[r], data.results[r])

Road House (1989) 4 2
The Equalizer 2 0 0
The Fast and the Furious: Tokyo Drift 0 2
Rampage (2018) 0 0


KeyError: 371

In [81]:
data.titles[186]

'The Meg'

NameError: name 'sim' is not defined

In [73]:
sim.to_csv('sim.csv')

In [74]:
data.to_csv('data_final.csv')

In [4]:
import requests
import ast

In [108]:
a = requests.get('https://api.themoviedb.org/searcg/movie/2?api_key=cc1ffc8a914b190fcb9a809741d325da')

In [120]:
df['originalTitle'][1]

'Le clown et ses chiens'

In [121]:
a = requests.get('https://api.themoviedb.org/3/search/movie?api_key=cc1ffc8a914b190fcb9a809741d325da&query=Le clown et ses chiens')

In [122]:
a.text

'{"page":1,"total_results":1,"total_pages":1,"results":[{"popularity":0.6,"id":16613,"video":false,"vote_count":2,"vote_average":4,"title":"The Clown and His Dogs","release_date":"1892-10-27","original_language":"en","original_title":"Le Clown et ses chiens","genre_ids":[16],"backdrop_path":null,"adult":false,"overview":"Short film of 300 individually painted images.","poster_path":"\\/qiJFROs4PDFGnIFYoSePaCN1it6.jpg"}]}'

In [130]:
import json
s = a.text
json_acceptable_string = s.replace("'", "\"")
d = json.loads(json_acceptable_string)

In [132]:
d = d['results'][0]

In [133]:
d

{'popularity': 0.6,
 'id': 16613,
 'video': False,
 'vote_count': 2,
 'vote_average': 4,
 'title': 'The Clown and His Dogs',
 'release_date': '1892-10-27',
 'original_language': 'en',
 'original_title': 'Le Clown et ses chiens',
 'genre_ids': [16],
 'backdrop_path': None,
 'adult': False,
 'overview': 'Short film of 300 individually painted images.',
 'poster_path': '/qiJFROs4PDFGnIFYoSePaCN1it6.jpg'}

In [134]:
d['original_title'], d['overview'], d['original_language'], d['release_date'], d['tagline'], d['id'], d['popularity']

KeyError: 'tagline'

In [4]:
df = pd.read_csv('data.tsv', sep='\t')

C:\Users\pbren\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
original_title = []
overview = []
original_language = []
release_date = []
tagline = []
movie_id = []
popularity = []

In [27]:
import json

In [5]:
df_movie = df[df.titleType == "movie"]
df_movie.reset_index(drop=True, inplace=True)

In [6]:
year = [0 if t == "\\N" else t for t in df_movie.startYear]

In [7]:
df_movie['startYear'] = year

C:\Users\pbren\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_movie['startYear'] = df_movie['startYear'].astype(int)
df_recmovie = df_movie[df_movie['startYear'] >= 1980]

C:\Users\pbren\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_recmovie.sort_values(by='startYear', ascending=False, inplace=True)

C:\Users\pbren\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
for t in tqdm(df_recmovie['originalTitle'][:10000]):
    url = str('https://api.themoviedb.org/3/search/movie?api_key=cc1ffc8a914b190fcb9a809741d325da&query=')+str(t)
    a = requests.get(url)
    try:
        d = json.loads(a.text)['results'][0]
        try:
            original_title.append(d['original_title'])
        except:
            original_title.append('NaN')
        try:
            overview.append(d['overview'])
        except:
            overview.append('NaN')
        try:
            original_language.append(d['original_language'])
        except:
            original_language.append('NaN')
        try:
            release_date.append(d['release_date'])
        except:
            release_date.append('NaN')
        try:
            tagline.append(d['tagline'])
        except:
            tagline.append('NaN')
        try:
            movie_id.append(d['movie_id'])
        except:
            movie_id.append('NaN')
        try:
            popularity.append(d['popularity'])
        except:
            popularity.append('NaN')
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [1:12:10<00:00,  2.35it/s]


In [11]:
collected_data = pd.DataFrame({'original_title':original_title, 'overview':overview,
                               'original_language':original_language, 'release_date':release_date, 'tagline':tagline,
                               'movie_id':movie_id, 'popularity':popularity})
collected_data.drop_duplicates(subset='original_title', inplace=True)
collected_data.reset_index(drop=True, inplace=True)

NameError: name 'original_title' is not defined

In [71]:
collected_data.drop("Plot unknown.")

,original_title,overview,original_language,release_date,tagline,movie_id,popularity
4,Untitled Marvel Project (3),Plot unknown.,en,2022-07-29,NaN,NaN,0.600
5,Untitled Disney Live Action Project,Plot unknown.,en,2023-02-17,NaN,NaN,0.600
80,Sherlock Holmes 3,Plot unknown.,en,2021-12-21,NaN,NaN,2.972
100,The Organ Donor,Plot unknown.,en,2020-05-15,NaN,NaN,0.959
403,Warkop DKI Reborn: Part 3,Plot unknown.,id,2019-09-12,NaN,NaN,3.303
1224,Lorong,Plot unknown.,id,2019-09-12,NaN,NaN,2.976
1597,Suwung,Plot unknown.,id,2019-07-18,NaN,NaN,0.600
2305,Ritual,Plot unknown.,id,2019-08-22,NaN,NaN,2.775
2431,Buya Hamka,Plot unknown.,id,,NaN,NaN,0.600
4121,Susi Susanti: Love All,Plot unknown.,id,2019-10-24,NaN,NaN,1.571


In [95]:
df.shape

(6052596, 9)

In [62]:
from autocomplete import models

In [63]:
import autocomplete

In [67]:
autocomplete.load()

training!
Error in opening pickle object. Training on default corpus text.
saving to: C:\Users\pbren\Anaconda3\lib\site-packages\autocomplete\models_compressed.pkl
done training!


True

In [77]:
autocomplete.predict('the', 'f')

[('furious', 5),
 ('fast', 3),
 ('fate', 2),
 ('future', 2),
 ('flies', 1),
 ('five', 1),
 ('first', 1),
 ('fugitive', 1),
 ('force', 1),
 ('final', 1)]

In [71]:
titles = ' '.join(data.titles)

In [72]:
models.train_models(titles)

saving to: C:\Users\pbren\Anaconda3\lib\site-packages\autocomplete\models_compressed.pkl


In [76]:
for t in data.titles:
    if 'the' in t:
        print(t)

Star Wars: Return of the Jedi
Harry Potter and the Sorcerer's Stone
Harry Potter and the Philosopher's Stone
The Treasure of the Sierra Madre
Snow White & the Huntsman
Lord of the Rings: The Two Towers
Lord of the Flies
Teenage Mutant Ninja Turtles II: The Secret of the Ooze
Jumanji: Welcome To the Jungle
Dr. Seuss' the Lorax
The Girl With the Dragon Tattoo
We're the Millers (2013)
The Fate of the Furious
Guardians of the Galaxy, Vol. 2
Star Wars: Attack of the Clones
Babe: Pig in the City
Jack the Giant Slayer
Pirates of the Caribbean: Dead Man's Chest
Into the Storm (2014)
Pirates of the Caribbean: At World's End
On the Job
The Fast and the Furious
Back to the Future Part III
Indiana Jones and the Temple of Doom
The Assassination of Jesse James By the Coward Robert Ford
Black Panther (2018)
War of the Worlds (2005)
The Lord of the Rings: The Return of the King (Extended Edition)
Sicario: Day of the Soldado
Ant-Man and the Wasp
The Chronicles of Narnia: The Voyage of the Dawn Treader


In [26]:
from gensim.models import Word2Vec, KeyedVectors

In [12]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [20]:
import torch
from pytorch_transformers import RobertaModel, RobertaConfig

In [23]:
roberta = RobertaModel.from_pretrained('roberta-base')

100%|███████████████████████████████████████████████████████████████| 501200538/501200538 [01:15<00:00, 12051570.86B/s]


In [25]:
model.wmdistance('I love men', 'I am gay')

ModuleNotFoundError: No module named 'pyemd'